![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# LAB | Connecting Python to SQL

<details>
  <summary>
   <h2>Learning Goals</h2>
  </summary>

  This lab allows you to practice and apply the concepts and techniques taught in class. 

  Upon completion of this lab, you will be able to:
  
- Write a Python script to connect to a relational database using the appropriate Python library and query it using SQL commands.

  <br>
  <hr> 

</details>

<details>
  <summary>
   <h2>Prerequisites</h2>
  </summary>

Before this starting this lab, you should have learnt about:

- Basic SQL queries
- Python and Pandas 
- Data Wrangling, which involves tasks such as grouping, aggregating, dealing with indexes, renaming columns, merging data and performing calculations.
 
  <br>
  <hr> 

</details>


## Introduction

Welcome to the Connecting Python to SQL lab!

In this lab, you will be working with the [Sakila](https://dev.mysql.com/doc/sakila/en/) database on movie rentals. Specifically, you will be practicing how to do basic SQL queries using Python. By connecting Python to SQL, you can leverage the power of both languages to efficiently manipulate and analyze large datasets. Throughout this lab, you will practice how to use Python to retrieve and manipulate data stored in the Sakila database using SQL queries. Let's get started!

## Challenge

In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity differ between months. To achieve this, follow these steps:

1. Establish a connection between Python and the Sakila database.

2. Write a Python function called `rentals_month` that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

	- `engine`: an object representing the database connection engine to be used to establish a connection to the Sakila database.
	- `month`: an integer representing the month for which rental data is to be retrieved.
	- `year`: an integer representing the year for which rental data is to be retrieved.

	The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

3. Develop a Python function called `rental_count_month` that takes the DataFrame provided by `rentals_month` as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year. 

	The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".


	*Hint: Consider making use of pandas [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)*

4. Create a Python function called `compare_rentals` that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. 
The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [1]:
# 1. Establish a connection between Python and the Sakila database.

import pandas as pd
import pymysql
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Retrieve database connection details from the .env file
username = os.getenv("my_sql_username")
password = os.getenv("my_sql_password")
host = os.getenv("my_sql_host", "localhost")  # Default to localhost when not set in .env
port = os.getenv("my_sql_port", 3306)  # Default MySQL port when not set in .env
db_name = os.getenv("my_sql_db_name", "sakila")  # This time connecting to bank, or set in .env

# Create the SQLAlchemy connection string
connection_string = f"mysql+pymysql://{username}:{password}@{host}:{port}/{db_name}"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

In [ ]:
# 2. Retrieve rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame.

def rentals_month(engine, month, year):
    df = pd.read_sql(f"SELECT * FROM rental WHERE month(rental_date) = {month} AND year(rental_date) = {year}", engine)
    return df

rentals_month(engine, 5, 2005)

In [3]:
# 3. rental_count_month

def rental_count_month(engine, month, year):
    return rentals_month(engine, month, year).groupby("customer_id").size().reset_index(name=f"rental_{month}_{year}")

df1 = rental_count_month(engine, 5, 2005)
df2 = rental_count_month(engine, 6, 2005)

In [ ]:
# 4. compare_rentals

def compare_rentals(engine, month1, year1, month2, year2):

    df1 = rental_count_month(engine, month1, year1)
    df2 = rental_count_month(engine, month2, year2)
    
    df = pd.merge(df1, df2, on="customer_id", how="outer")
    
    df.fillna(0, inplace=True)
    
    df["difference"] = df[f"rental_{month1}_{year1}"] - df[f"rental_{month2}_{year2}"]
    
    return df

compare_rentals(engine, 6, 2005, 5, 2005)